In [2]:
from scipy.stats import qmc
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import shutil
import os
from sklearn.metrics import mean_squared_error
from math import sqrt
import subprocess
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")



In [3]:
#reading in data with relative paths so that it can be used by others who clone repository from github
script_dir = os.getcwd()

#Calibration Folder
calibration_folder_path = os.path.abspath(os.path.join(script_dir, '..', '..', 'calibration'))

#June Source Folder
August_calibration_path = os.path.abspath(os.path.join(script_dir, '..', '..', 'calibration', 'August'))

#Temperature Observations folder
observed_temp_path = os.path.abspath(os.path.join(script_dir, '..', '..', 'observations', 'TemperatureDataObs'))





In [4]:
#Making standard lhc to calibrate two dates

# sampler = qmc.LatinHypercube(d=12)
# sample = sampler.random(n=1000)

# l_bounds = [1.4, 6.20e-3, 0.25, 0.01, 0.60, 1.00e-9, 1.00e-9, 9.0, .5, .1, 0, 0]
# u_bounds = [1.6, 6.80e-3, 0.4, 0.04, 1.00, 4.00e-9, 4.00e-9, 13.0, .8, .2, 2, 1]
# lhs = qmc.scale(sample, l_bounds, u_bounds)

#  # Output Latin hypercube to OptRMSE file
# lhs_df = pd.DataFrame(lhs, columns=['thermal conductivity', 'thermal diffusivity', 'hypo thickness', 'hypo %',
#                                'sediment porosity', 'wind function a', 'wind function b', "accretion temp", "tree canopy", "grass canopy", "tree overhang", "grass overhang"])#, 'Overall RMSE']).to_csv(
# lhs_df.to_csv(os.path.join(calibration_folder_path,'hypercube_paramters1000.csv'))


In [4]:
lhs = pd.read_csv(os.path.join(calibration_folder_path,'hypercube_parameters1000.csv'),  index_col=[0])
lhs_df = pd.DataFrame(lhs, columns=['thermal conductivity', 'thermal diffusivity', 'hypo thickness', 'hypo %',
                               'sediment porosity', 'wind function a', 'wind function b', "accretion temp", "tree canopy", "grass canopy", "tree overhang", "grass overhang"])
lhs.iloc[1,8]
lhs_df

,thermal conductivity,thermal diffusivity,hypo thickness,hypo %,sediment porosity,wind function a,wind function b,accretion temp,tree canopy,grass canopy,tree overhang,grass overhang
0,1.589739,0.006693,0.277969,0.012962,0.607604,2.700286e-09,2.792647e-09,10.830394,0.559755,0.108751,1.530267,0.826332
1,1.593486,0.006308,0.386867,0.012137,0.646473,3.447290e-09,3.282401e-09,9.383360,0.758393,0.144381,1.231513,0.398736
2,1.447345,0.006293,0.305921,0.034472,0.650536,1.089236e-09,2.251392e-09,12.565471,0.594054,0.184032,0.335779,0.052414
3,1.406989,0.006761,0.383032,0.020680,0.873685,2.820607e-09,1.102571e-09,10.007236,0.751449,0.183003,0.301451,0.196670
4,1.570616,0.006592,0.393133,0.035451,0.625397,2.270726e-09,1.609290e-09,12.030047,0.696049,0.157197,0.638785,0.205271
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.567029,0.006743,0.283044,0.013645,0.927248,2.399722e-09,1.637229e-09,9.165179,0.629259,0.160541,0.552024,0.739461
996,1.563420,0.006584,0.258387,0.021050,0.685840,1.061351e-09,2.431400e-09,11.579462,0.599685,0.135954,1.639880,0.487997
997,1.450943,0.006686,0.389896,0.023276,0.992971,2.260998e-09,2.914381e-09,9.810255,0.593766,0.153070,0.418899,0.693607
998,1.441378,0.006238,0.260855,0.020295,0.973506,3.217560e-09,1.227746e-09,10.998451,0.637334,0.120816,0.842850,0.466131


In [6]:
### Large calibration run
source_folder = August_calibration_path
os.chdir(source_folder)  # Change the current working directory

control_file = pd.read_csv(os.path.join(source_folder, 'HeatSource_Control.csv'))
morph_file = pd.read_csv(os.path.join(source_folder, "inputs", 'WFK23_morphology.csv'))
accretion_file = pd.read_csv(os.path.join(source_folder, "inputs", 'WFK23_accretion_notrib.csv'))
lccode_file = pd.read_csv(os.path.join(source_folder, "inputs", 'WFK23_lccode.csv'))
                          
cc_start = control_file.loc[7]['VALUE']
cc_start = pd.to_datetime(cc_start)
cc_end = control_file.loc[8]['VALUE']
cc_end = pd.to_datetime(cc_end)


obs = pd.read_csv(os.path.join(observed_temp_path, 'HourlySummer23Master_riverkm.csv'))
obs["datetime"] = pd.to_datetime(obs["datetime"])
obs_clip = obs[(obs["datetime"]>=cc_start) & (obs["datetime"]<cc_end + pd.DateOffset(1))]
obs_clip = obs_clip.dropna(axis=1)
obs_clip = obs_clip.set_index('datetime')
#obs_clip.columns = [float(col) for col in obs_clip.columns[2:]]
obs_clip.columns = [float(col) for col in obs_clip.columns]
obs_clip.drop(19.3, axis=1, inplace=True)

#obs_clip.drop(19.2, axis=1, inplace=True)

obs_tw = obs_clip.to_numpy()
overall_rmse_list = []
nse_list = []
#columns =  ["17.5", "16.1", "12.2", "10.7", "7.2", "1.2", "0", "overall rmse", "nse"]
rmse_df = pd.DataFrame(columns=[*obs_clip.columns, "overall rmse", "nse"])
loop_counter = 0
for n in range(1000):
    print(n)
    # Populate BHF parameters
    control_file.loc[25, 'VALUE'] = lhs.iloc[n, 5]  # wind function a
    control_file.loc[26, 'VALUE'] = lhs.iloc[n, 6]  # wind function b
    #control_file.loc[28, 'VALUE'] = lhs[n, 7] # deep alluvium temp
    control_file.to_csv(os.path.join(source_folder, 'HeatSource_Control.csv'), index = False)

    # Populate flow parameters
    morph_file['SED_THERMAL_CONDUCTIVITY'] = lhs.iloc[n, 0]  # thermal conductivity
    morph_file['SED_THERMAL_DIFFUSIVITY'] = lhs.iloc[n, 1]  # thermal diffusivity
    morph_file['SED_HYPORHEIC_THICKNESSS'] = lhs.iloc[n, 2]  # hypo thickness
    morph_file['HYPORHEIC_PERCENT'] = lhs.iloc[n, 3]  # hypo %
    morph_file['POROSITY'] = lhs.iloc[n, 4]  # sediment porosity
    #morph_file['MANNINGS_n']=lhs[n, 7] # Mannings N
    morph_file.to_csv(os.path.join(source_folder, "inputs", 'WFK23_morphology.csv'), index = False)
    #LCCode file
    lccode_file["CANOPY"].iloc[1:] = lhs.iloc[n, 8] #closure for tree polygons
    lccode_file["OVERHANG"].iloc[1:] = lhs.iloc[n, 10] #overhang for tree polygons
    lccode_file["CANOPY"].iloc[:1] = lhs.iloc[n, 9] #closure for grass polygons
    lccode_file["OVERHANG"].iloc[:1] = lhs.iloc[n, 11] #overhang for grass polygons
    lccode_file.to_csv(os.path.join(source_folder, "inputs", 'WFK23_lccode.csv'), index = False)
    # Populate accretion parameterss
    accretion_file['TEMPERATURE'] = lhs.iloc[n, 7] #accretion temp
    accretion_file.to_csv(os.path.join(source_folder, "inputs", 'WFK23_accretion_notrib.csv'), index = False)

    # Run HS9
    #os.system('D:/Ben_wd/THESIS/heatsource/WestFork/ModelRuns/Hayes/runs/latinhypercubetest/hs9_run_temperature.py')
    #!hs run -t
    try:
        
        # Run the 'hs' command with 'run -t' in the new folder, suppressing output
        subprocess.run(['hs', 'run', '-t'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        
        print(str(n) + ' successful runs')
    except:
        # Print a message indicating which run failed and continue to the next iteration
        print('run #: ' + str(n) + ' failed.')
        continue

    # Get data
    
    sim_tw_df = pd.read_csv(os.path.join(source_folder, 'outputs', 'Temp_H2O.csv'), skiprows=6,
             usecols=range(1, 187))
    sim_tw_df.columns = [float(col) for col in sim_tw_df.columns]
    common_columns = set(sim_tw_df.columns) & set(obs_clip.columns)
    sim_tw_df = sim_tw_df[obs_clip.columns]

    sim_tw = sim_tw_df.to_numpy()
#     lhs_df = pd.DataFrame(lhs, columns=['thermal conductivity', 'thermal diffusivity', 'hypo thickness', 'hypo %',
#                                'sediment porosity', 'wind function a', 'wind function b', "accretion temp", "tree canopy", "grass canopy", "tree overhang", "grass overhang"])#, 'Overall RMSE']).to_csv(
        
    # Calculate RMSE if sim_tw is valid
    if sim_tw.shape == obs_tw.shape:
        # Calculate Overall RMSE for the current row
        rmse_values = np.sqrt(np.mean((sim_tw - obs_tw) ** 2, axis=0))

        # Calculate Overall RMSE for the current row
        overall_rmse = sum(rmse_values)/(len(obs_clip.columns)-1)

        # Calculate NSE for the current row
        mean_observed = np.mean(obs_tw)
        numerator = np.sum((obs_tw - sim_tw)**2)
        denominator = np.sum((obs_tw - mean_observed)**2)
        nse = 1 - (numerator / denominator)

        # Add RMSE, Overall RMSE, and NSE to the DataFrame as a new row
        row_data = np.concatenate([rmse_values, [overall_rmse, nse]])
        #print("Row data before adding to DataFrame:", row_data)
        rmse_df = pd.concat([rmse_df, pd.DataFrame([row_data], columns=rmse_df.columns)], ignore_index=True)

    else:
        # Append a placeholder value if sim_tw has 0 rows
        zero_row = np.zeros((1, len(rmse_df.columns)))
        rmse_df = pd.concat([rmse_df, pd.DataFrame(zero_row, columns=rmse_df.columns)], ignore_index=True)

    # Calculate RMSE
    # overall_rmse = sqrt(mean_squared_error(obs_tw, sim_tw))
    # overall_rmse_list.append(overall_rmse)
    loop_counter += 1
    print(loop_counter)
    lhs_df = pd.concat([lhs, rmse_df], axis=1)

    #added this line after calibration crashed and needed to write to an existing lhs_df
    #lhs_df = pd.concat([lhs.iloc[n], rmse_df], axis=1)
    
    # Write the current loop's data to the CSV file
    lhs_df.to_csv(os.path.join(source_folder, 'OptRMSE1000_August.csv'), index=False)
    lhs_df = pd.DataFrame(lhs, columns=['thermal conductivity', 'thermal diffusivity', 'hypo thickness', 'hypo %',
                               'sediment porosity', 'wind function a', 'wind function b', "accretion temp", "tree canopy", "grass canopy", "tree overhang", "grass overhang"])#, 'Overall RMSE']).to_csv(
        
    

#lhs_df = pd.concat([lhs_df, rmse_df], axis=1)

#lhs_df.to_csv(os.path.join(source_folder, 'OptRMSE.csv'), index=False)


0
0 successful runs
1
1
1 successful runs
2
2
2 successful runs
3
3
3 successful runs
4
4
4 successful runs
5
5
5 successful runs
6
6
6 successful runs
7
7
7 successful runs
8
8
8 successful runs
9
9
9 successful runs
10
10
10 successful runs
11
11
11 successful runs
12
12
12 successful runs
13
13
13 successful runs
14
14
14 successful runs
15
15
15 successful runs
16
16
16 successful runs
17
17
17 successful runs
18
18
18 successful runs
19
19
19 successful runs
20
20
20 successful runs
21
21
21 successful runs
22
22
22 successful runs
23
23
23 successful runs
24
24
24 successful runs
25
25
25 successful runs
26
26
26 successful runs
27
27
27 successful runs
28
28
28 successful runs
29
29
29 successful runs
30
30
30 successful runs
31
31
31 successful runs
32
32
32 successful runs
33
33
33 successful runs
34
34
34 successful runs
35
35
35 successful runs
36
36
36 successful runs
37
37
37 successful runs
38
38
38 successful runs
39
39
39 successful runs
40
40
40 successful runs
41
41
4